In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error
from sklearn.metrics import median_absolute_error, mean_squared_error, mean_squared_log_error

from scipy.optimize import minimize
import statsmodels.tsa.api as smt
import statsmodels.api as sm

from tqdm import tqdm_notebook

from itertools import product

def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
data = pd.read_excel('Train Set.xlsx')
data

,Region_Name,Month 1,Month 2,Month 3,Month 4,Month 5,Month 6,Month 7,Month 8,Month 9,Month 10,Month 11,Month 12,Month 13,Month 14,Month 15,Month 16,Month 17,Month 18,Month 19,Month 20,Month 21,Month 22,Month 23,Month 24,Month 25,Month 26,Month 27,Month 28,Month 29,Month 30,Month 31,Month 32,Month 33,Month 34,Month 35,Month 36,Month 37,Month 38,Month 39,Month 40,Month 41,Month 42,Month 43,Month 44,Month 45,Month 46,Month 47,Month 48,Month 49,Month 50,Month 51,Month 52,Month 53,Month 54,Month 55,Month 56,Month 57,Month 58,Month 59,Month 60,Month 61,Month 62,Month 63,Month 64,Month 65,Month 66,Month 67,Month 68,Month 69,Month 70,Month 71,Month 72
0,Region 1,0.154325,0.122901,0.127583,0.501557,0.103551,0.051983,0.140272,0.157586,0.182546,0.181801,-0.013496,0.213958,0.202585,0.256381,0.327656,0.277944,0.200623,0.141673,0.133457,0.092708,0.124225,0.199409,0.211584,0.256088,0.046845,0.050224,0.044738,0.049602,0.185713,0.141933,0.032244,0.101614,0.202953,0.185604,0.083222,0.097904,0.241120,0.264636,0.253286,0.422999,0.464442,0.316720,0.224314,0.338289,0.378352,0.520046,0.391280,0.314726,0.270200,0.362765,0.376728,0.394986,0.347692,0.351117,0.375120,0.289649,0.372379,0.398122,0.362610,0.263766,0.275622,0.252173,0.336913,0.321828,0.321895,0.337018,0.121581,0.234171,0.294693,0.373754,0.333642,0.242176
1,Region 2,0.119708,0.119831,0.112970,0.326049,0.098852,0.041502,0.116759,0.136964,0.178476,0.162621,-0.017723,0.227958,0.181563,0.247475,0.488062,0.290772,0.196146,0.139117,0.108014,0.091059,0.122529,0.202530,0.206454,0.243916,0.030216,0.057625,0.037567,0.047221,0.175260,0.121241,-0.009893,0.066981,0.191505,0.170095,0.057635,0.075968,0.216270,0.233841,0.245432,0.407445,0.437820,0.332397,0.238877,0.318503,0.366288,0.502219,0.360538,0.294045,0.241370,0.334915,0.359444,0.378980,0.306557,0.350775,0.342869,0.272034,0.344482,0.372061,0.318767,0.221102,0.221851,0.191496,0.283412,0.302669,0.306597,0.327412,0.124563,0.218367,0.275179,0.344997,0.300503,0.215147
2,Region 3,0.178236,0.165497,0.190754,0.321733,0.203126,0.048345,0.180064,0.176990,0.224173,0.223810,0.070306,0.314094,0.253627,0.317007,0.493548,0.356269,0.301312,0.295783,0.213944,0.225560,0.199628,0.295161,0.301142,0.369074,0.148873,0.148648,0.141286,0.176595,0.298081,0.247686,0.073358,0.167071,0.369198,0.273285,0.166522,0.232556,0.324115,0.337240,0.328820,0.488371,0.452943,0.299717,0.276886,0.353269,0.391376,0.575888,0.412834,0.404446,0.315161,0.413679,0.436428,0.452330,0.355533,0.397592,0.429922,0.315471,0.425693,0.437532,0.401876,0.320588,0.310557,0.285627,0.331988,0.377643,0.384990,0.362927,0.230516,0.263643,0.296751,0.400269,0.360406,0.335923
3,Region 4,0.225016,0.220602,0.224240,0.330916,0.268934,0.171322,0.243672,0.219831,0.242115,0.227575,0.068486,0.300100,0.260220,0.339176,0.408422,0.367631,0.374176,0.385319,0.200377,0.308989,0.264775,0.300382,0.328013,0.395487,0.204112,0.210327,0.230464,0.254440,0.428539,0.309254,0.155241,0.227263,0.471488,0.294128,0.162800,0.191990,0.303951,0.360914,0.397259,0.534679,0.493992,0.373266,0.296719,0.367477,0.400149,0.625336,0.415876,0.373174,0.305040,0.419634,0.408965,0.453475,0.325201,0.445034,0.414192,0.308857,0.381268,0.432136,0.366376,0.264211,0.271887,0.248869,0.315252,0.329762,0.352229,0.368514,0.192878,0.228902,0.291502,0.364874,0.311379,0.271102
4,Region 5,0.318976,0.316835,0.328486,0.483001,0.380898,0.292074,0.268395,0.326809,0.344086,0.278257,0.187874,0.588132,0.376303,0.429040,0.484136,0.547735,0.458580,0.588121,0.336758,0.398324,0.404609,0.351204,0.421711,0.620374,0.446481,0.354543,0.378399,0.423216,0.478113,0.386218,0.323202,0.331631,0.547656,0.383964,0.226795,0.392175,0.430364,0.439188,0.421460,0.672617,0.556797,0.527481,0.380318,0.429014,0.463007,0.671208,0.487795,0.500731,0.328601,0.450884,0.471953,0.506876,0.388457,0.417775,0.470368,0.363075,0.410614,0.482598,0.408525,0.381367,0.336923,0.288830,0.366314,0.392924,0.370159,0.422587,0.282631,0.289498,0.368923,0.416527,0.372523,0.392835
...,...,...,...,...,...,...,...,...,...,...,...,..

In [ ]:
df = data.transpose()
df.columns = df.iloc[0]
df.drop(df.index[0],inplace=True)
df.rename(columns={'Region_Name':'Month'},inplace=True)
df

Region_Name,Region 1,Region 2,Region 3,Region 4,Region 5,Region 6,Region 7,Region 8,Region 9,Region 10,Region 11,Region 12,Region 13,Region 14,Region 15,Region 16,Region 17,Region 18,Region 19,Region 20,Region 21,Region 22,Region 23,Region 24,Region 25,Region 26,Region 27,Region 28,Region 29,Region 30,Region 31,Region 32,Region 33,Region 34,Region 35,Region 36,Region 37,Region 38,Region 39,Region 40,...,Region 3876,Region 3877,Region 3878,Region 3879,Region 3880,Region 3881,Region 3882,Region 3883,Region 3884,Region 3885,Region 3886,Region 3887,Region 3888,Region 3889,Region 3890,Region 3891,Region 3892,Region 3893,Region 3894,Region 3895,Region 3896,Region 3897,Region 3898,Region 3899,Region 3900,Region 3901,Region 3902,Region 3903,Region 3904,Region 3905,Region 3906,Region 3907,Region 3908,Region 3909,Region 3910,Region 3911,Region 3912,Region 3913,Region 3914,Region 3915
Month 1,0.154325,0.119708,0.178236,0.225016,0.318976,0.0499222,0.219802,0.0690548,0.186616,0.0535246,0.190577,0.158609,0.165217,0.069157,0.157302,0.0723155,0.0247574,0.180025,0.206183,0.0506291,0.212532,0.137973,0.0753769,0.172759,3.58705,0.315296,0.8732,0.157227,0.282547,0.164005,0.0834449,0.0123568,0.245894,0.0658875,0.154048,0.342258,0.312587,0.105453,0.100788,0.0860421,...,0.444992,0.276426,0.495871,0.754557,0.317987,2.44748,1.55207,1.12038,0.908094,0.878383,1.04391,1.03511,16.2653,0.430467,0.492921,0.281649,0.599147,1.5003,0.363861,0.315941,0.344324,1.01054,0.38755,0.196002,2.23129,0.353811,0.785468,0.290261,0.468493,0.408514,0.154372,0.338739,0.253205,0.233119,0.199864,3.40048,0.374751,0.463527,1.64151,0.307089
Month 2,0.122901,0.119831,0.165497,0.220602,0.316835,0.0823366,0.175329,0.0596021,0.185539,0.0502944,0.147166,0.141044,0.159698,0.0461758,0.159635,0.0711093,0.0507293,0.172872,0.182795,0.0677798,0.163495,0.135268,0.0826447,0.152321,3.00115,0.291904,0.717435,0.128892,0.217566,0.224851,0.0744534,0.041007,0.201204,0.0895518,0.128339,0.267466,0.272249,0.100428,0.142518,0.0974825,...,0.709842,0.441313,0.69015,1.02227,0.45799,3.47795,1.71948,1.32542,0.81273,1.16898,1.67155,1.29567,20.4655,0.657171,0.559615,0.375383,0.623699,1.81589,0.472307,0.594319,0.634195,1.12895,0.56814,0.2921,2.52732,0.489,0.963027,0.449931,0.59772,0.489851,0.180818,0.424401,0.336452,0.290894,0.328968,3.99957,0.478654,0.571315,1.64994,0.451604
Month 3,0.127583,0.11297,0.190754,0.22424,0.328486,0.0831819,0.178562,0.0718142,0.207523,0.0603539,0.13125,0.162525,0.175699,0.0567053,0.511734,0.0904734,0.0548434,0.195426,0.231616,0.0864705,0.221174,0.185897,0.163404,0.22026,3.02542,0.327416,0.70496,0.444618,0.294857,0.219886,0.44278,0.0767134,0.237413,0.128469,0.214407,0.344732,0.524357,0.322261,0.158039,0.229757,...,0.53461,0.402494,0.662206,1.03332,0.441139,3.41602,1.85386,1.34586,1.05939,1.11318,1.19387,1.32601,20.5364,0.684776,0.668832,0.328184,0.618134,1.71731,0.527924,0.539337,0.531636,1.24704,0.516666,0.282652,2.6949,0.460567,0.957489,0.42822,0.617278,0.519781,0.407456,0.485308,0.369691,0.301755,0.27136,3.30845,0.501871,0.644025,2.11045,0.448063
Month 4,0.501557,0.326049,0.321733,0.330916,0.483001,0.156701,0.245769,0.123014,0.364449,0.115983,0.198793,0.2167,0.238456,0.145198,0.665948,0.155938,0.155571,0.271052,0.285454,0.188782,0.53927,0.180723,0.174683,0.282689,2.34532,0.605174,0.600299,0.375266,0.313095,0.235294,0.472638,0.16151,0.261674,0.254538,0.200919,0.377825,0.485539,0.217263,0.191939,0.21638,...,0.489195,0.407036,0.673595,0.95477,0.451388,2.84043,1.50885,1.09043,0.93453,1.09348,1.02261,1.07085,15.6966,0.592267,0.601639,0.399364,0.589935,1.57969,0.497742,0.531691,0.532295,1.12694,0.521146,0.364811,2.43294,0.484782,0.965312,0.413396,0.607717,0.568379,0.403261,0.473134,0.3766,0.358181,0.447163,2.6367,0.481275,0.636814,1.64235,0.464692
Month 5,0.103551,0.0988525,0.203126,0.268934,0.380898,0.0132889,0.139204,0.0788197,0.165327,0.0728055,0.116737,0.122395,0.143661,0.0427863,0.136179,0.0565386,0.031122,0.10978,0.12033,0.0271493,0.14148,0.0974298,0.0686056,0.117808,2.01571,0.234

In [ ]:
df.isna().sum().sum()

0

In [ ]:
import itertools
p = d = q = range(0, 2)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]
print('Examples of parameter for SARIMA...')
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))

Examples of parameter for SARIMA...
SARIMAX: (0, 0, 1) x (0, 0, 1, 12)
SARIMAX: (0, 0, 1) x (0, 1, 0, 12)
SARIMAX: (0, 1, 0) x (0, 1, 1, 12)
SARIMAX: (0, 1, 0) x (1, 0, 0, 12)


In [ ]:
evaluated_data = pd.read_csv('evaluated_data3.csv')

In [ ]:
evaluated_data

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1.1,Region 1,Region 2,Region 3,Region 4,Region 5,Region 6,Region 7,Region 8,Region 9,Region 10,Region 11,Region 12,Region 13,Region 14,Region 15,Region 16,Region 17,Region 18,Region 19,Region 20,Region 21,Region 22,Region 23,Region 24,Region 25,Region 26,Region 27,Region 28,Region 29,Region 30,...,Region 3490,Region 3491,Region 3492,Region 3493,Region 3494,Region 3495,Region 3496,Region 3497,Region 3498,Region 3499,Region 3500,Region 3501,Region 3502,Region 3503,Region 3504,Region 3505,Region 3506,Region 3507,Region 3508,Region 3509,Region 3510,Region 3511,Region 3512,Region 3513,Region 3514,Region 3515,Region 3516,Region 3517,Region 3518,Region 3519,Region 3520,Region 3521,Region 3522,Region 3523,Region 3524,Region 3525,Region 3526,Region 3527,Region 3528,Region 3529
0,0,0,0,0,0,0,0,0,0,0,0.517301,0.492149,0.549738,0.411588,0.331036,0.474639,0.700946,0.536144,0.422265,0.469925,0.584476,0.439094,0.364509,0.478135,0.499001,0.451424,0.458245,0.544922,0.496880,0.429247,0.527439,0.585806,0.483933,0.562956,0.383433,0.382476,0.553390,0.439743,0.539712,0.646762,...,0.868349,0.348558,0.675881,0.508801,0.640060,0.447480,0.474816,0.643374,0.371280,0.646182,0.621525,0.452186,0.327700,0.800547,0.434704,0.213382,0.106345,0.462043,0.737446,0.602238,0.406986,0.346120,0.919082,0.677077,0.553475,0.362223,0.725353,0.842937,0.606539,0.779547,0.859011,0.712365,0.817825,0.420384,0.557232,0.471496,0.964130,0.615946,0.579099,0.734121
1,1,1,1,1,1,1,1,1,1,1,0.534972,0.501310,0.551330,0.426970,0.328557,0.498653,0.658362,0.553508,0.441389,0.496942,0.584476,0.445722,0.392568,0.499946,0.513363,0.476169,0.489399,0.544922,0.503498,0.463472,0.524600,0.576984,0.502016,0.562956,0.274237,0.463315,0.500737,0.452591,0.539712,0.628632,...,0.950245,0.382652,0.885205,0.546991,0.635062,0.575926,0.591243,0.663724,0.531400,0.864631,0.663209,0.543015,0.493033,0.791415,0.499471,0.338691,0.265861,0.567280,0.850567,0.627224,0.443220,0.486884,1.062405,0.689852,0.585398,0.396803,0.742221,0.946961,0.761487,0.916986,0.869545,0.729229,0.839700,0.462599,0.636401,0.611073,0.991909,0.751038,0.691909,0.854102
2,2,2,2,2,2,2,2,2,2,2,0.543166,0.516219,0.551879,0.431927,0.335697,0.507934,0.666844,0.559795,0.447291,0.505783,0.584476,0.447224,0.402311,0.508036,0.518135,0.485446,0.500409,0.544922,0.505009,0.478187,0.519751,0.574239,0.508855,0.562956,0.261060,0.535385,0.493458,0.456368,0.539712,0.623997,...,0.960192,0.368302,0.893266,0.566365,0.668070,0.712127,0.739663,0.672523,0.742914,0.851383,0.620405,0.756355,0.584037,0.787447,0.534211,0.691100,0.480799,0.761104,0.884326,0.706801,0.469124,0.657330,1.019551,0.697305,0.597997,0.429510,0.733304,0.951694,0.812424,0.931185,0.824257,0.684821,0.796054,0.505779,0.697098,0.775791,0.931084,0.853540,0.723532,0.827418
3,3,3,3,3,3,3,3,3,3,3,0.546967,0.519715,0.552069,0.433525,0.349890,0.511522,0.679758,0.562072,0.449112,0.508677,0.584476,0.447564,0.405694,0.511036,0.581388,0.488925,0.504300,0.544922,0.505354,0.484513,0.515522,0.573385,0.511441,0.562956,0.246781,0.652064,0.459121,0.457478,0.539712,0.622812,...,0.988813,0.354937,0.900627,0.576194,0.699814,0.741613,0.761624,0.676328,0.726794,0.876582,0.588506,0.926730,0.716971,0.785724,0.562708,0.656317,0.653352,0.819723,0.965430,0.810549,0.513666,0.741224,0.975844,0.701652,0.602970,0.458572,0.710529,1.006692,0.890802,1.001561,0.729788,0.603385,0.819374,0.509083,0.662296,0.893347,0.836570,0.832989,0.814509,0.858194
4,4,4,4,4,4,4,4,4,4,4,0.548729,0.521119,0.552135,0.434040,0.325881,0.512908,0.657066,0.562896,0.449674,0.509624,0.584476,0.447641,0.406869,0.512149,0.568178,0.490230,0.505675,0.544922,0.505433,0.487234,0.511736,0.573119,0.512419,0.562956,0.242215,0.530343,0.481016,0.457804,0.539712,0.622509,...,0.881002,0.278383,0.727404,0.581180,0.572378,0.538103,0.562975,0.677974,0.590855,0.696027,0.472762,0.650

In [ ]:
y_pred_complete = []
# index = -1
# evaluated_data = pd.DataFrame()
def complete_model():
  # index = 0
  for x in df.loc[:,"Region 3530":"Region 3915"].columns:
    aic_list = []
    param_list = []
    param_seasonal_list = []
    for param in pdq:
        for param_seasonal in seasonal_pdq:
          mod = sm.tsa.statespace.SARIMAX(df_scaled_table[x].astype(float),order=param,seasonal_order=param_seasonal,enforce_stationarity=False,enforce_invertibility=False)
          results = mod.fit()
          # print('ARIMA{}x{}12 - AIC:{}'.format(param,param_seasonal,results.aic))
          aic_list.append(results.aic)
          param_list.append(param)
          param_seasonal_list.append(param_seasonal)
  
    min_value = aic_list.index(min(aic_list))
    mod = sm.tsa.statespace.SARIMAX(df_scaled_table[x].astype(float),
                                order=param_list[min_value],
                                seasonal_order=param_seasonal_list[min_value],
                                enforce_stationarity=False,
                                enforce_invertibility=False)
    results = mod.fit()
    y_pred = results.forecast(steps=16)
    y_pred_complete.append(list(y_pred))
    evaluated_data[x] = list(y_pred)
    print(x)


In [ ]:
complete_model()

Region 3530
Region 3531
Region 3532
Region 3533
Region 3534
Region 3535
Region 3536
Region 3537
Region 3538
Region 3539
Region 3540
Region 3541
Region 3542
Region 3543
Region 3544
Region 3545
Region 3546
Region 3547
Region 3548
Region 3549
Region 3550
Region 3551
Region 3552
Region 3553
Region 3554
Region 3555
Region 3556
Region 3557
Region 3558
Region 3559
Region 3560
Region 3561
Region 3562
Region 3563
Region 3564
Region 3565
Region 3566
Region 3567
Region 3568
Region 3569
Region 3570
Region 3571
Region 3572
Region 3573
Region 3574
Region 3575
Region 3576
Region 3577
Region 3578
Region 3579
Region 3580
Region 3581
Region 3582
Region 3583
Region 3584
Region 3585
Region 3586
Region 3587
Region 3588
Region 3589
Region 3590
Region 3591
Region 3592
Region 3593
Region 3594
Region 3595
Region 3596
Region 3597
Region 3598
Region 3599
Region 3600
Region 3601
Region 3602
Region 3603
Region 3604
Region 3605
Region 3606
Region 3607
Region 3608
Region 3609
Region 3610
Region 3611
Region 3612
Regi

In [ ]:
evaluated_data

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1,Region 1,Region 2,Region 3,Region 4,Region 5,Region 6,Region 7,Region 8,Region 9,Region 10,Region 11,Region 12,Region 13,Region 14,Region 15,Region 16,Region 17,Region 18,Region 19,Region 20,Region 21,Region 22,Region 23,Region 24,Region 25,Region 26,Region 27,Region 28,Region 29,Region 30,Region 31,...,Region 3490,Region 3491,Region 3492,Region 3493,Region 3494,Region 3495,Region 3496,Region 3497,Region 3498,Region 3499,Region 3500,Region 3501,Region 3502,Region 3503,Region 3504,Region 3505,Region 3506,Region 3507,Region 3508,Region 3509,Region 3510,Region 3511,Region 3512,Region 3513,Region 3514,Region 3515,Region 3516,Region 3517,Region 3518,Region 3519,Region 3520,Region 3521,Region 3522,Region 3523,Region 3524,Region 3525,Region 3526,Region 3527,Region 3528,Region 3529
0,0,0,0,0,0,0,0,0,0,0.517301,0.492149,0.549738,0.411588,0.331036,0.474639,0.700946,0.536144,0.422265,0.469925,0.584476,0.439094,0.364509,0.478135,0.499001,0.451424,0.458245,0.544922,0.496880,0.429247,0.527439,0.585806,0.483933,0.562956,0.383433,0.382476,0.553390,0.439743,0.539712,0.646762,0.531384,...,0.868349,0.348558,0.675881,0.508801,0.640060,0.447480,0.474816,0.643374,0.371280,0.646182,0.621525,0.452186,0.327700,0.800547,0.434704,0.213382,0.106345,0.462043,0.737446,0.602238,0.406986,0.346120,0.919082,0.677077,0.553475,0.362223,0.725353,0.842937,0.606539,0.779547,0.859011,0.712365,0.817825,0.420384,0.557232,0.471496,0.964130,0.615946,0.579099,0.734121
1,1,1,1,1,1,1,1,1,1,0.534972,0.501310,0.551330,0.426970,0.328557,0.498653,0.658362,0.553508,0.441389,0.496942,0.584476,0.445722,0.392568,0.499946,0.513363,0.476169,0.489399,0.544922,0.503498,0.463472,0.524600,0.576984,0.502016,0.562956,0.274237,0.463315,0.500737,0.452591,0.539712,0.628632,0.524711,...,0.950245,0.382652,0.885205,0.546991,0.635062,0.575926,0.591243,0.663724,0.531400,0.864631,0.663209,0.543015,0.493033,0.791415,0.499471,0.338691,0.265861,0.567280,0.850567,0.627224,0.443220,0.486884,1.062405,0.689852,0.585398,0.396803,0.742221,0.946961,0.761487,0.916986,0.869545,0.729229,0.839700,0.462599,0.636401,0.611073,0.991909,0.751038,0.691909,0.854102
2,2,2,2,2,2,2,2,2,2,0.543166,0.516219,0.551879,0.431927,0.335697,0.507934,0.666844,0.559795,0.447291,0.505783,0.584476,0.447224,0.402311,0.508036,0.518135,0.485446,0.500409,0.544922,0.505009,0.478187,0.519751,0.574239,0.508855,0.562956,0.261060,0.535385,0.493458,0.456368,0.539712,0.623997,0.533060,...,0.960192,0.368302,0.893266,0.566365,0.668070,0.712127,0.739663,0.672523,0.742914,0.851383,0.620405,0.756355,0.584037,0.787447,0.534211,0.691100,0.480799,0.761104,0.884326,0.706801,0.469124,0.657330,1.019551,0.697305,0.597997,0.429510,0.733304,0.951694,0.812424,0.931185,0.824257,0.684821,0.796054,0.505779,0.697098,0.775791,0.931084,0.853540,0.723532,0.827418
3,3,3,3,3,3,3,3,3,3,0.546967,0.519715,0.552069,0.433525,0.349890,0.511522,0.679758,0.562072,0.449112,0.508677,0.584476,0.447564,0.405694,0.511036,0.581388,0.488925,0.504300,0.544922,0.505354,0.484513,0.515522,0.573385,0.511441,0.562956,0.246781,0.652064,0.459121,0.457478,0.539712,0.622812,0.535834,...,0.988813,0.354937,0.900627,0.576194,0.699814,0.741613,0.761624,0.676328,0.726794,0.876582,0.588506,0.926730,0.716971,0.785724,0.562708,0.656317,0.653352,0.819723,0.965430,0.810549,0.513666,0.741224,0.975844,0.701652,0.602970,0.458572,0.710529,1.006692,0.890802,1.001561,0.729788,0.603385,0.819374,0.509083,0.662296,0.893347,0.836570,0.832989,0.814509,0.858194
4,4,4,4,4,4,4,4,4,4,0.548729,0.521119,0.552135,0.434040,0.325881,0.512908,0.657066,0.562896,0.449674,0.509624,0.584476,0.447641,0.406869,0.512149,0.568178,0.490230,0.505675,0.544922,0.505433,0.487234,0.511736,0.573119,0.512419,0.562956,0.242215,0.530343,0.481016,0.457804,0.539712,0.622509,0.539304,...,0.881002,0.278383,0.727404,0.581180,0.572378,0.538103,0.562975,0.677974,0.590855,0.69602

In [ ]:
evaluated_data.to_csv('evaluated_data3.csv')

In [ ]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.tsa.api import VAR

In [ ]:
model = VAR(df.astype(float))
model_fit = model.fit()

In [ ]:
pred = model_fit.forecast(model_fit.y, steps=16)
pred.shape

(16, 3915)

In [ ]:
var_data = pd.DataFrame()
i = 0
for x in df.columns:
  var_data[x] = pred[:,i]
  i += 1
# pred_region_1 = pred[:,0]
# pred_region_1

In [ ]:
var_data.drop(0,axis=0,inplace=True)

In [ ]:
var_data.to_csv('var_data.csv')

In [ ]:
mean_absolute_percentage_error(test_set['Region 1'],pred_region_1)

23.85213229344951

In [ ]:
# Augmented Dickey-Fuller Test (ADF Test)/unit root test
from statsmodels.tsa.stattools import adfuller
def adf_test(ts, signif=0.05):
    dftest = adfuller(ts, autolag='AIC')
    adf = pd.Series(dftest[0:4], index=['Test Statistic','p-value','# Lags','# Observations'])
    for key,value in dftest[4].items():
       adf['Critical Value (%s)'%key] = value
    print (adf)
    
    p = adf['p-value']
    if p <= signif:
        print(f" Series is Stationary")
    else:
        print(f" Series is Non-Stationary")
#apply adf test on the series
adf_test(train_set['Region 4'])
# adf_test(df_train["realcons"])

Test Statistic          -4.524288
p-value                  0.000178
# Lags                   0.000000
# Observations          59.000000
Critical Value (1%)     -3.546395
Critical Value (5%)     -2.911939
Critical Value (10%)    -2.593652
dtype: float64
 Series is Stationary


In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(df)
df_scaled = scaler.transform(df)

In [ ]:
df_scaled_table = pd.DataFrame()
i = 0
for x in df.columns:
  df_scaled_table[x] = df_scaled[:,i]
  i += 1

In [ ]:
df_scaled_table.to_csv('scaled_data.csv')

In [ ]:
from statsmodels.tsa.stattools import adfuller
dftest = adfuller(df['Region 1'], autolag = 'AIC')
print("1. ADF : ",dftest[0])
print("2. P-Value : ", dftest[1])
print("3. Num Of Lags : ", dftest[2])
print("4. Num Of Observations Used For ADF Regression and Critical Values Calculation :", dftest[3])
print("5. Critical Values :")
for key, val in dftest[4].items():
    print("\t",key, ": ", val)

1. ADF :  -4.068787494799374
2. P-Value :  0.0010901999186739824
3. Num Of Lags :  0
4. Num Of Observations Used For ADF Regression and Critical Values Calculation : 71
5. Critical Values :
	 1% :  -3.526004646825607
	 5% :  -2.9032002348069774
	 10% :  -2.5889948363419957


In [ ]:
sarimax_model = sm.tsa.statespace.SARIMAX(df['Region 1'].astype(float),order=(1,1,1),seasonal_order=(1,1,1,30))

In [ ]:
res = sarimax_model.fit(disp=False)

ValueError: ignored